In [ ]:
import os
import webbrowser
from datetime import datetime
from openpyxl import Workbook
import requests
import html5lib
import numpy as np
import pandas as pd
import unicodedata
import re
from bs4 import BeautifulSoup

import helper

import warnings
warnings.filterwarnings('ignore')


def parse_and_trim(content, content_type):
    if content_type == "HTML":
        soup = BeautifulSoup(content, 'html.parser')
    else:
        soup = BeautifulSoup(content, 'html.parser')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

In [ ]:
headers = {
    'User-Agent': 'APOLLO INVESTMENT CORPORATION',
    'User-Agent': 'MIDCAP FINANCIAL INVESTMENT CORPORATION'
}
filing_links = pd.read_excel('../MFIC__sec_filing_links.xlsx')
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")
filing_links.head(5)

In [ ]:
for index, (url, reporting_date) in enumerate(zip(filing_links['url'], filing_links['Reporting date'])):
    pass

In [ ]:
response = helper.get_response(
    url=filing_links['url'].iloc[1], headers=headers)
filing_links['url'].iloc[1]
content = helper.get_content(response)

In [ ]:
content = helper.get_content(response)

In [ ]:
consolidated_schedule_regex = re.compile(
    r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')

for tag in content.find_all(text=consolidated_schedule_regex):
    print("Tag:", tag)
    print("Find_next:", tag.find_next())
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    print("Next_date:", re.search(date_regex_pattern1, tag.find_next().text))
    print("next:", tag.next)
    print("Next next: ", tag.find_next().next.next.next.text)
    print("next sib: ", tag.find_next_sibling())
    print("\n")

In [ ]:
def extract_tables(soup_content, qtr_date):
    master_table = None
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern1, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern1, tag.next.text)
        print(date_str)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                html_table = tag.find_next('table')
                new_table = pd.read_html(
                    html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table

In [ ]:
for index, (url, reporting_date) in enumerate(zip(filing_links['url'], filing_links['Reporting date'])):

    try:
        content = helper.get_content(
            helper.get_response(url=url, headers=headers))
    except Exception as e:
        print(f'failed to get the content: {e}')
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')

    for tag in content.find_all(text=consolidated_schedule_regex):
        print("Tag:", tag)
        # print("Find_next:", tag.find_next())
        date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
        print("Next_date:", re.search(date_regex_pattern1, tag.find_next().text))
        # print("next:", tag.next)
        # print("Next next: ", tag.find_next().next.next.next.text)
        # print("next sib: ", tag.find_next_sibling())
        print("\n")

for index, (url, reporting_date) in enumerate(zip(filing_links['url'], filing_links['Reporting date'])):

    try:
        content = helper.get_content(
            helper.get_response(url=url, headers=headers))
    except Exception as e:
        print(f'failed to get the content: {e}')

    master_table = extract_tables(content, reporting_date)
    master_table.to_csv(
        '../MT_csv_file/'+reporting_date.replace(',', '')+'.csv')

In [89]:
data_frames = []
data_frames_shapes = []


def extract_tables(soup_content, qtr_date):
    master_table = None
    print("now doing", qtr_date)
    consolidated_schedule_regex = re.compile(
        r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')
    date_regex_pattern = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    # date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    for tag in soup_content.find_all(text=re.compile(consolidated_schedule_regex)):
        date_str = re.search(date_regex_pattern, tag.find_next().text)
        if date_str is None:
            date_str = re.search(date_regex_pattern, tag.next.text)
        if date_str is None:
            date_str = re.search(date_regex_pattern,
                                 tag.find_next().next.next.next.text)
        if date_str is not None:
            date_str = str(date_str.group(1))
            date_str = unicodedata.normalize('NFKD', date_str)
            qtr_date_cleaned = qtr_date.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            date_str_cleaned = date_str.replace(',', '').replace(
                ' ', '').replace('\n', '').lower()
            print(date_str, qtr_date_cleaned, date_str_cleaned)

            if qtr_date_cleaned == date_str_cleaned:
                if qtr_date_cleaned in ["march312015", "march312014", "june302014", "june302015", "september302014", "september302015", "december312014"]:
                    html_table = tag.find_parent('table')
                else:
                    html_table = tag.find_next('table')
                if html_table is not None:
                    new_table = pd.read_html(
                        html_table.prettify(), keep_default_na=False, skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)

                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)

            # master_table = master_table.replace('N/A', 'No Value')
    try:
        master_table = master_table.applymap(
            lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
        master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
            r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    except Exception as e:
        print(f'{e}')
    print(master_table.shape)
    data_frames.append(master_table)
    data_frames_shapes.append(master_table.shape)
    return master_table

In [90]:
path = '/Users/fuadhassan/Desktop/BDC_RA/MFIC/FeedBack_Master_tables_MFIC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
for index, (url, reporting_date) in enumerate(zip(filing_links['url'], filing_links['Reporting date'])):

    try:
        content = helper.get_content(
            helper.get_response(url=url, headers=headers))
    except Exception as e:
        print(f'failed to get the content: {e}')

    master_table = extract_tables(content, reporting_date)
    master_table.to_csv(
        '../MT_csv_file/'+reporting_date.replace(',', '')+'.csv')
    master_table.to_excel(
        writer, sheet_name=reporting_date.replace(',', ''), index=False)
    writer.book .save(path)
writer.close()

now doing September 30, 2023
September 30, 2023 september302023 september302023
December 31, 2022 september302023 december312022
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
September 30, 2023 september302023 september302023
Sept

In [ ]:
try:
    content = helper.get_content(
        helper.get_response(url=url, headers=headers))
except Exception as e:
    print(f'failed to get the content: {e}')
master_table = extract_tables(content, reporting_date)
master_table.to_excel("test.xlsx")

In [ ]:
reporting_date = 'March 31, 2015'
url = helper.get_file_url(reporting_date, filing_links=filing_links)
url

In [ ]:
consolidated_schedule_regex = re.compile(
    r'(?i)^\s*.*\s*SCHEDULE(S|)\s+OF\s+INVESTMENTS\s*.*\s*$')

for tag in content.find_all(string=consolidated_schedule_regex):
    print("Tag:", tag)
    print("Find_next:", tag.find_next())
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    print("Next_date:", re.search(date_regex_pattern1, tag.find_next().text))
    print("next:", tag.next)
    print("Next next: ", tag.find_next().next.next.next.text)
    print("next table: ", tag.find_parent('table'))
    print("\n")

march312015, march312014, june302014, june302015, september302014, september302015, december312014